In [2]:
import pymysql #데이터베이스 연결
import numpy as np #수치 계산
from scipy.spatial.distance import euclidean, cosine #거리 계산

In [3]:
# user_record와 album 테이블에서 필요한 데이터를 가져오는 함수
def fetch_records_and_albums(connection):
    with connection.cursor() as cursor: #데이터베이스 커서가 생성되고, 블록이 끝나면 자동으로 닫힘
        cursor.execute("SELECT user_id, album_id FROM user_record") # user_record 테이블에서 user_id와 album_id를 가져오기 위한 SQL 쿼리 실행
        records = cursor.fetchall()  # 쿼리 결과를 records 변수에 저장
        
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """) # album 테이블에서 id와 여러 특성 값을 가져오기 위한 SQL 쿼리 실행
        albums = cursor.fetchall()  # 쿼리 결과를 albums 변수에 저장
    return records, albums  # 가져온 데이터를 반환

In [5]:
def main():
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    ) # MySQL 연결

    try:
        records, albums = fetch_records_and_albums(connection) # user_record, album 데이터 가져옴

        album_dict = {album[0]: np.array(album[1:]) for album in albums} # 앨범의 id를 키로 하고, 특성 값을 값으로 하는 딕셔너리를 생성

        for user_id, album_id in records: # 각 사용자가 청취한 앨범에 대해 반복
            current_features = album_dict[album_id]  # 현재 앨범의 특성 값

            closest_album_id_euclidean = None  # 유클리드 거리 기준 가장 가까운 앨범 id 초기화
            closest_distance_euclidean = float('inf')  # 초기 가장 작은 유클리드 거리 설정

            closest_album_id_cosine = None  # 코사인 유사도 기준 가장 가까운 앨범 id 초기화
            closest_distance_cosine = float('inf')  # 초기 가장 작은 코사인 거리 설정
            
            for other_album_id, features in album_dict.items(): # 나머지 앨범들과 현재 앨범의 유사도 계산
                if other_album_id != album_id:  # 동일한 앨범은 비교하지 않음
                    # 유클리드 거리 계산
                    euclidean_distance = euclidean(current_features, features)
                    if euclidean_distance < closest_distance_euclidean:
                        closest_distance_euclidean = euclidean_distance
                        closest_album_id_euclidean = other_album_id

                    # 코사인 유사도 계산
                    cosine_distance = cosine(current_features, features)
                    if cosine_distance < closest_distance_cosine:
                        closest_distance_cosine = cosine_distance
                        closest_album_id_cosine = other_album_id

            # 결과 출력
            print(f"current_album_id: {album_id}, Closest album by Cosine similarity: {closest_album_id_cosine}")


    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        connection.close()  # 데이터베이스 연결 닫기

if __name__ == "__main__":
    main()

current_album_id: 1, Closest album by Cosine similarity: 15
current_album_id: 6, Closest album by Cosine similarity: 17
current_album_id: 29, Closest album by Cosine similarity: 5


record에 있는 각 앨범마다 유사한 값을 비교하는게 아닌 record에 담긴 앨범 특징의 평균값과 유사한 곡들을 추출하는 것으로 변경

In [ ]:
import pymysql
import numpy as np
from scipy.spatial.distance import euclidean, cosine

# user_record와 album 테이블에서 필요한 데이터를 가져오는 함수
def fetch_records_and_albums(connection):
    with connection.cursor() as cursor:
        # user_record 테이블에서 user_id와 album_id를 가져오기 위한 SQL 쿼리 실행
        cursor.execute("SELECT user_id, album_id FROM user_record")
        records = cursor.fetchall()  # 쿼리 결과를 records 변수에 저장
        
        # album 테이블에서 id와 여러 특성 값을 가져오기 위한 SQL 쿼리 실행
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """)
        albums = cursor.fetchall()  # 쿼리 결과를 albums 변수에 저장
    return records, albums  # 가져온 데이터를 반환


def calculate_average_features(user_records, album_dict): # user_records에 포함된 album_id에 대해 평균 특성 벡터를 계산
    feature_sums = np.zeros(len(next(iter(album_dict.values()))))  # 첫 번째 앨범의 특성 벡터 길이만큼 0으로 초기화
    count = 0

    for user_id, album_id in user_records:
        feature_sums += album_dict[album_id]
        count += 1

    if count == 0:
        return feature_sums
    else:
        return feature_sums / count  # 평균 특성 벡터 계산


# 메인 함수
def main():
    # MySQL 데이터베이스에 연결 설정
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    )

    try:
        # 데이터베이스에서 레코드와 앨범 데이터를 가져옴
        records, albums = fetch_records_and_albums(connection)

        # 앨범의 id를 키로 하고, 특성 값을 값으로 하는 딕셔너리를 생성
        album_dict = {album[0]: np.array(album[1:]) for album in albums}

        # user_record에 있는 모든 user_id에 대해 평균 특성 벡터 계산
        average_features = calculate_average_features(records, album_dict)

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        connection.close()  # 데이터베이스 연결 닫기

if __name__ == "__main__":
    main()
